In [1]:
from sklearn.model_selection import train_test_split
import numpy as np

images = np.load("images.npy")
labels = np.load("labels.npy")

X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.1, random_state=42)

# X_train = np.load("images_train.npy")
# y_train = np.load("labels_train.npy")
# X_test = np.load("images_test.npy")
# y_test = np.load("labels_test.npy")

X_train_flatten = X_train.reshape(X_train.shape[0], -1).T
X_test_flatten = X_test.reshape(X_test.shape[0], -1).T
y_train = y_train.reshape(1, y_train.shape[0])
y_test = y_test.reshape(1, y_test.shape[0])

train_set_x = X_train_flatten / 255.
test_set_x = X_test_flatten / 255.

print(train_set_x.shape)


(3072, 9000)


In [2]:
def initialize_with_zeros(dim):
    w = np.zeros([dim, 1], dtype="float64")
    b = 0.0
    return w, b


def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s


def propagate(w, b, X, Y):
    m = X.shape[1]
    A = sigmoid(np.dot(w.T, X) + b)
    cost = -(np.sum(Y * np.log(A) + (1.0 - Y) * np.log(1.0 - A))) / m

    dw = np.dot(X, (A - Y).T) / m
    db = np.sum(A - Y) / m

    cost = np.squeeze(np.array(cost))
    grads = {"dw": dw,
             "db": db}

    return grads, cost


In [3]:
def gradient_decent(w, b, X, Y, num_iterations=100, learning_rate=0.009):
    costs = []

    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)

        dw = grads["dw"]
        db = grads["db"]

        w = w - learning_rate * dw
        b = b - learning_rate * db


        if i % 100 == 0:
            costs.append(cost)
            print("Cost after iteration %i: %f" % (i, cost))

    params = {"w": w,
              "b": b}

    grads = {"dw": dw,
             "db": db}

    return params, grads, costs


In [4]:
def predict(w, b, X):
    m = X.shape[1]
    w = w.reshape(X.shape[0], 1)
    Y_prediction = np.zeros((1, m))

    A = sigmoid(np.dot(w.T, X) + b)

    for i in range(A.shape[1]):
        if A[0, i] > 0 and A[0, i] < 0.5:
            Y_prediction[0, i] = 0
        else:
            Y_prediction[0, i] = 1

    return Y_prediction


In [5]:
c = []
def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5):
    w, b = initialize_with_zeros(X_train.shape[0])
    params, grads, costs = gradient_decent(
        w, b, X_train, Y_train, num_iterations, learning_rate)
    w = params["w"]
    b = params["b"]

    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    print("train accuracy: {} %".format(
        100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(
        100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    c.append(costs)
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test,
         "Y_prediction_train": Y_prediction_train,
         "w": w,
         "b": b,
         "learning_rate": learning_rate,
         "num_iterations": num_iterations}

    return d


In [6]:
logistic_regression_model = model(train_set_x, y_train, test_set_x, y_test, num_iterations=10000, learning_rate=0.01)


Cost after iteration 0: 0.693147
Cost after iteration 100: 0.668503
Cost after iteration 200: 0.663777
Cost after iteration 300: 0.661196
Cost after iteration 400: 0.659372
Cost after iteration 500: 0.657910
Cost after iteration 600: 0.656658
Cost after iteration 700: 0.655542
Cost after iteration 800: 0.654524
Cost after iteration 900: 0.653581
Cost after iteration 1000: 0.652698
Cost after iteration 1100: 0.651865
Cost after iteration 1200: 0.651075
Cost after iteration 1300: 0.650323
Cost after iteration 1400: 0.649604
Cost after iteration 1500: 0.648915
Cost after iteration 1600: 0.648254
Cost after iteration 1700: 0.647618
Cost after iteration 1800: 0.647006
Cost after iteration 1900: 0.646415
Cost after iteration 2000: 0.645844
Cost after iteration 2100: 0.645292
Cost after iteration 2200: 0.644757
Cost after iteration 2300: 0.644239
Cost after iteration 2400: 0.643737
Cost after iteration 2500: 0.643249
Cost after iteration 2600: 0.642775
Cost after iteration 2700: 0.642314
Cost

In [15]:
c

[]

In [7]:
def relu(Z):
    """
    Implement the RELU function.

    Arguments:
    Z -- Output of the linear layer, of any shape

    Returns:
    A -- Post-activation parameter, of the same shape as Z
    cache -- a python dictionary containing "A" ; stored for computing the backward pass efficiently
    """

    A = np.maximum(0, Z)

    assert(A.shape == Z.shape)

    cache = Z
    return A, cache


In [8]:
def sigmoid(Z):
    """
    Implements the sigmoid activation in numpy
    
    Arguments:
    Z -- numpy array of any shape
    
    Returns:
    A -- output of sigmoid(z), same shape as Z
    cache -- returns Z as well, useful during backpropagation
    """

    A = 1/(1+np.exp(-Z))
    cache = Z

    return A, cache


In [ ]:
def sigmoid_backward(dA, cache):
    """
    Implement the backward propagation for a single SIGMOID unit.

    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently

    Returns:
    dZ -- Gradient of the cost with respect to Z
    """

    Z = cache

    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)

    assert (dZ.shape == Z.shape)

    return dZ


In [ ]:
def relu_backward(dA, cache):
    """
    Implement the backward propagation for a single RELU unit.

    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently

    Returns:
    dZ -- Gradient of the cost with respect to Z
    """

    Z = cache
    dZ = np.array(dA, copy=True)  # just converting dz to a correct object.

    # When z <= 0, you should set dz to 0 as well.
    dZ[Z <= 0] = 0

    assert (dZ.shape == Z.shape)

    return dZ
